### Importing Libraries

In [1]:
# import necessary libraries
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
from keras.models import load_model

# create an object of WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# importing the GL Bot corpus file for pre-processing

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open("mentalhealth.json",errors="ignore").read()
intents = json.loads(data_file)

### Data Pre-Processing

In [2]:
# preprocessing the json data
# tokenization
nltk.download('punkt')
nltk.download('wordnet')
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# lemmatize, lower each word and remove duplicates

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

# creating a pickle file to store the Python objects which we will use while predicting
pickle.dump(words,open('words.pkl','wb')) 
pickle.dump(classes,open('classes.pkl','wb'))

129 documents
102 classes ['ADHD', 'CBD_oil', 'Causes', 'MSP', 'Psychiatric_AdvanceDirective', 'affects_whom', 'alcohol_or_drug_use', 'antidepressants', 'antisocial_personality_disorder', 'balanced_perspectives', 'binge_eating', 'borderline_personality_disorder', 'brain_injury', 'cannabis', 'child_mental_health', 'clinical_trials', 'contact_professsionals', 'cure_mentalhealth', 'cyclothymic_disorder', 'dealing_with_people', 'definition', 'depression', 'diagnosis', 'difference_between_CBT_and_DBT', 'difference_between_anxiety_and_an_anxiety_disorder', 'difference_between_anxiety_and_stress', 'difference_between_dissociative_identity_disorder_and_schizophrenia?', 'difference_between_mental_health_and_mental_illness', 'difference_between_psychiatrist_and_psychologist', 'difference_between_psychosis_and_schizophrenia', 'difference_between_psychotherapy_and_counselling?', 'difference_between_sadness_and_depression', 'drinking_too_much', 'evidence_on_vaping', 'find_a_psychiatrist', 'find_a_p

### Creating Training Data

In [4]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
   
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# shuffle features and converting it into numpy arrays
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

print("Training data created")

Training data created


### Creating NN Model

In [5]:
# Create NN model to predict the responses
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot.h5', hist) # we will pickle this model to use in the future
print("\n")
print("*"*50)
print("\nModel Created Successfully!")

Epoch 1/200
26/26 [==============================] - 1s 3ms/step - loss: 4.6331 - accuracy: 0.0155
Epoch 2/200
26/26 [==============================] - 0s 3ms/step - loss: 4.6119 - accuracy: 0.0078
Epoch 3/200
26/26 [==============================] - 0s 3ms/step - loss: 4.5602 - accuracy: 0.0310
Epoch 4/200
26/26 [==============================] - 0s 3ms/step - loss: 4.4845 - accuracy: 0.0543
Epoch 5/200
26/26 [==============================] - 0s 3ms/step - loss: 4.4438 - accuracy: 0.0698
Epoch 6/200
26/26 [==============================] - 0s 3ms/step - loss: 4.3762 - accuracy: 0.0853
Epoch 7/200
26/26 [==============================] - 0s 3ms/step - loss: 4.3258 - accuracy: 0.1008
Epoch 8/200
26/26 [==============================] - 0s 3ms/step - loss: 4.2337 - accuracy: 0.1085
Epoch 9/200
26/26 [==============================] - 0s 3ms/step - loss: 4.0465 - accuracy: 0.1008
Epoch 10/200
26/26 [==============================] - 0s 3ms/step - loss: 4.0402 - accuracy: 0.1240
Epoch 11/

26/26 [==============================] - 0s 3ms/step - loss: 0.8323 - accuracy: 0.7597
Epoch 83/200
26/26 [==============================] - 0s 3ms/step - loss: 1.0685 - accuracy: 0.6589
Epoch 84/200
26/26 [==============================] - 0s 3ms/step - loss: 1.0175 - accuracy: 0.6512
Epoch 85/200
26/26 [==============================] - 0s 3ms/step - loss: 0.9302 - accuracy: 0.6977
Epoch 86/200
26/26 [==============================] - 0s 3ms/step - loss: 0.9182 - accuracy: 0.6744
Epoch 87/200
26/26 [==============================] - 0s 3ms/step - loss: 1.1218 - accuracy: 0.6822
Epoch 88/200
26/26 [==============================] - 0s 3ms/step - loss: 0.8160 - accuracy: 0.7752
Epoch 89/200
26/26 [==============================] - 0s 2ms/step - loss: 1.1452 - accuracy: 0.6512
Epoch 90/200
26/26 [==============================] - 0s 3ms/step - loss: 0.9687 - accuracy: 0.7287
Epoch 91/200
26/26 [==============================] - 0s 3ms/step - loss: 1.0515 - accuracy: 0.6667
Epoch 92/200


26/26 [==============================] - 0s 3ms/step - loss: 0.7256 - accuracy: 0.7674
Epoch 164/200
26/26 [==============================] - 0s 3ms/step - loss: 0.6499 - accuracy: 0.8140
Epoch 165/200
26/26 [==============================] - 0s 3ms/step - loss: 0.8457 - accuracy: 0.7209
Epoch 166/200
26/26 [==============================] - 0s 3ms/step - loss: 0.8835 - accuracy: 0.7752
Epoch 167/200
26/26 [==============================] - 0s 3ms/step - loss: 0.7783 - accuracy: 0.7364
Epoch 168/200
26/26 [==============================] - 0s 3ms/step - loss: 0.6202 - accuracy: 0.8062
Epoch 169/200
26/26 [==============================] - 0s 3ms/step - loss: 0.7887 - accuracy: 0.7209
Epoch 170/200
26/26 [==============================] - 0s 3ms/step - loss: 1.0123 - accuracy: 0.7132
Epoch 171/200
26/26 [==============================] - 0s 3ms/step - loss: 0.7256 - accuracy: 0.7209
Epoch 172/200
26/26 [==============================] - 0s 3ms/step - loss: 0.9857 - accuracy: 0.7287
Epoc

### Repeating all the above steps and using the pickled model for the prediction of responses

In [7]:
# load the saved model file
model = load_model('chatbot.h5')
intents = json.loads(open("mentalhealth.json",errors="ignore").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [8]:
def clean_up_sentence(sentence):

    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):

    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)

    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
               
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
   
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    error = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>error]
    
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [9]:
# function to get the response from the model

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

# function to predict the class and get the response

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [10]:
# function to start the chat bot which will continue till the user type 'end'

def start_chat():
    print("Bot: This is Sophie! Your Personal Assistant.\n\n")
    while True:
        inp = str(input()).lower()
        if inp.lower()=="end":
            break
        if inp.lower()== '' or inp.lower()== '*':
            print('Please re-phrase your query!')
            print("-"*50)
        else:
            print(f"Bot: {chatbot_response(inp)}"+'\n')
            print("-"*50)

## Chatting with BOT using the Command Line Option

In [ ]:
# start the chat bot
start_chat()

Bot: This is Sophie! Your Personal Assistant.


hello
Bot: Good to see you again

--------------------------------------------------
hi
Bot: Good to see you again

--------------------------------------------------
i am sad
Bot: Hi there, how can I help?

--------------------------------------------------
what is mental illness
Bot: Symptoms of mental health disorders vary depending on the type and severity of the condition.

--------------------------------------------------
what is mental health
Bot: We all have mental health which is made up of our beliefs, thoughts, feelings and behaviours.

--------------------------------------------------
how can i get help
Bot: It is important to be as involved and engaged in the treatment process as possible.

--------------------------------------------------
i need money 
Bot: My pleasure

--------------------------------------------------
what are my treatment options 
Bot: Just as there are different types of medications for physical illne

## Chatting with BOT using the Tkinter App

#### **For running the Tkinter GUI you have to download this notebook in ipynb format and run using jupyter notebook in your local machine/pc because in google colab, you cannot run Tkinter apps.**

In [ ]:
import random
import tkinter as tk
from tkinter import *

root=tk.Tk()
filename="Chat Bot"
root.title(f"Chat Bot")
root.geometry('500x400')
root.resizable(False, False)
message=tk.StringVar()

chat_win=Frame(root,bd=1,bg='white',width=50,height=8)
chat_win.place(x=6,y=6,height=300,width=488)

textcon=tk.Text(chat_win,bd=1,bg='white',width=50,height=8)
textcon.pack(fill="both",expand=True)

mes_win=Entry(root,width=30,xscrollcommand=True,textvariable=message)
mes_win.place(x=6,y=310,height=60,width=380)
mes_win.focus()

textcon.config(fg='black')
textcon.tag_config('usr',foreground='black')
textcon.insert(END,"Bot: This is Sophie! Your Personal Assistant.\n\n")
mssg=mes_win.get()

exit_list = ['exit','break','quit','see you later','chat with you later','end the chat','bye','ok bye']

def greet_res(text):
    text=text.lower()
    bot_greet=['hi','hello','hola','hey','howdy']
    usr_greet=['hi','hey','hello','hola','greetings','wassup','whats up']
    for word in text.split():
        if word in usr_greet:
            return random.choice(bot_greet)

def send_msz(event=None):
    usr_input = message.get()
    usr_input = usr_input.lower()
    textcon.insert(END, f'You: {usr_input}'+'\n','usr')
    if usr_input in exit_list:
        textcon.config(fg='black')
        textcon.insert(END,"Bot: Ok bye! Chat with you later\n")
        return root.destroy()
    else:
        textcon.config(fg='black')
        if greet_res(usr_input) != None:
            lab=f"Bot: {greet_res(usr_input)}"+'\n'
            textcon.insert(END,lab)
            mes_win.delete(0,END)
        else:
            lab = f"Bot: {chatbot_response(usr_input)}"+'\n'
            textcon.insert(END,lab)
            mes_win.delete(0,END)

button_send=Button(root,text='Send',bg='dark green',activebackground='grey',command=send_msz,width=12,height=5,font=('Arial'))
button_send.place(x=376,y=310,height=60,width=110)
root.bind('<Return>', send_msz,button_send)
root.mainloop()

### END